Реалізація децентралізованого протоколу

Інформація про квантові стани грошей розподіляється між кількома вузлами (учасниками системи).

Вузли голосують за автентичність купюри на основі своїх даних.

Завдяки квантовим принципам (суперпозиція, неможливість клонування) зловмисники не можуть підробити купюру.

In [5]:
#Кожен учасник мережі (вузол) отримує частину інформації про квантову купюру.
import random

class QuantumMoney:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.bases = [random.choice(['+', 'x']) for _ in range(num_qubits)]
        self.states = [random.choice([0, 1]) for _ in range(num_qubits)]

    def distribute_keys(self, num_nodes):
        """Розподіл інформації між вузлами."""
        shares = []
        for _ in range(num_nodes):
            indices = random.sample(range(self.num_qubits), k=self.num_qubits // num_nodes)
            share = {'indices': indices, 'bases': [self.bases[i] for i in indices], 'states': [self.states[i] for i in indices]}
            shares.append(share)
        return shares

# Генеруємо квантові гроші
money = QuantumMoney(12)
print("Оригінальні базиси:", money.bases)
print("Оригінальні стани:", money.states)

# Розподіл серед 3 вузлів
shares = money.distribute_keys(3)
for i, share in enumerate(shares, start=1):
    print(f"Ключ вузла {i}:", share)


Оригінальні базиси: ['+', 'x', '+', '+', '+', '+', 'x', 'x', '+', '+', '+', '+']
Оригінальні стани: [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
Ключ вузла 1: {'indices': [7, 2, 1, 3], 'bases': ['x', '+', 'x', '+'], 'states': [1, 1, 0, 1]}
Ключ вузла 2: {'indices': [10, 2, 9, 3], 'bases': ['+', '+', '+', '+'], 'states': [0, 1, 0, 1]}
Ключ вузла 3: {'indices': [3, 0, 5, 10], 'bases': ['+', '+', '+', '+'], 'states': [1, 1, 1, 0]}


In [6]:
#Кожен вузол отримує частину даних і голосує, чи купюра автентична.
class Node:
    def __init__(self, share):
        self.share = share

    def validate(self, provided_bases, provided_states):
        """Перевіряє частину квантових грошей."""
        correct = 0
        for i, index in enumerate(self.share['indices']):
            if self.share['bases'][i] == provided_bases[index]:  # Збіг базисів
                if self.share['states'][i] == provided_states[index]:  # Збіг станів
                    correct += 1
        return correct / len(self.share['indices'])

# Перевірка кожним вузлом
provided_bases = ['+', 'x', '+', 'x', '+', 'x', '+', 'x', '+', 'x', '+', 'x']  # Випадкові базиси
provided_states = [0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1]  # Випадкові стани

nodes = [Node(share) for share in shares]
votes = [node.validate(provided_bases, provided_states) for node in nodes]

print("Результати вузлів:", votes)

# Консенсус
threshold = 0.7  # Порог точності
if all(vote >= threshold for vote in votes):
    print("Купюра визнана автентичною!")
else:
    print("Купюра не автентична.")


Результати вузлів: [0.25, 0.0, 0.0]
Купюра не автентична.


Протокол з iнтерактивною перевiркою

Кубіти створюються з випадковими базисами (+ або x) і станами (0 або 1).

Банк надсилає список випадкових базисів для перевірки.
                                                               
Відповіді власника:
Якщо базиси збігаються з оригінальними, власник повертає справжні стани.
Якщо базиси не збігаються, власник вимірює у неправильному базисі, через що відповіді стають випадковими.

Банк порівнює відповіді власника з оригінальними станами.
Результат визначається як частка правильних відповідей.

In [7]:
import random

class QuantumMoney:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.bases = [random.choice(['+', 'x']) for _ in range(num_qubits)]
        self.states = [random.choice([0, 1]) for _ in range(num_qubits)]

    def interactive_check(self):
        """Інтерактивна перевірка з банком."""
        # Банк надсилає запити (випадкові базиси)
        bank_bases = [random.choice(['+', 'x']) for _ in range(self.num_qubits)]
        print("Банк запитує базиси:", bank_bases)

        # Власник відповідає на основі запитів банку
        responses = []
        for i in range(self.num_qubits):
            if bank_bases[i] == self.bases[i]:  # Якщо базиси співпадають
                responses.append(self.states[i])
            else:
                responses.append(random.choice([0, 1]))  # Випадковий стан
        print("Відповіді власника:", responses)

        # Банк перевіряє відповіді
        correct = 0
        for i in range(self.num_qubits):
            if bank_bases[i] == self.bases[i] and responses[i] == self.states[i]:
                correct += 1

        accuracy = correct / self.num_qubits
        print(f"Рівень точності перевірки: {accuracy * 100:.2f}%")
        return accuracy


# Створення купюри
money = QuantumMoney(10)
print("Оригінальні базиси:", money.bases)
print("Оригінальні стани:", money.states)

# Інтерактивна перевірка
accuracy = money.interactive_check()
if accuracy > 0.8:  # Порог точності для успішного проходження
    print("Купюра автентична!")
else:
    print("Купюра не автентична!")


Оригінальні базиси: ['+', '+', '+', '+', '+', '+', 'x', 'x', 'x', 'x']
Оригінальні стани: [0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
Банк запитує базиси: ['x', 'x', 'x', '+', '+', 'x', 'x', 'x', 'x', '+']
Відповіді власника: [1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
Рівень точності перевірки: 50.00%
Купюра не автентична!


Реалізація протоколу Візнера

1. Генерація купюри

In [8]:
import random

class WiesnerQuantumMoney:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.bases = [random.choice(['+', 'x']) for _ in range(num_qubits)]
        self.states = [random.choice([0, 1]) for _ in range(num_qubits)]

    def get_qubits(self):
        """Повертає квантові стани купюри (базиси та стани)"""
        return self.bases, self.states

2. Перевірка купюри

In [9]:
class Bank:
    def __init__(self, bases, states):
        """Зберігає базиси та стани купюри для перевірки"""
        self.bases = bases
        self.states = states

    def verify(self, provided_bases, provided_states):
        """Перевіряє відповідність базисів та станів"""
        correct = 0
        for i in range(len(self.bases)):
            if self.bases[i] == provided_bases[i]:  # Базиси збігаються
                if self.states[i] == provided_states[i]:  # Стани збігаються
                    correct += 1
        accuracy = correct / len(self.bases)
        print(f"Рівень точності перевірки: {accuracy * 100:.2f}%")
        return accuracy > 0.8  # Порог точності для успішної перевірки

3. Власник вимірює купюру

In [10]:
def owner_measurement(bases, states):
    """Власник вимірює квантові стани"""
    provided_bases = [random.choice(['+', 'x']) for _ in bases]  # Випадкові базиси
    provided_states = []
    for i in range(len(bases)):
        if provided_bases[i] == bases[i]:  # Вимірювання у відповідному базисі
            provided_states.append(states[i])
        else:
            provided_states.append(random.choice([0, 1]))  # Випадковий стан
    return provided_bases, provided_states

4. Демонстрація протоколу

In [11]:
# Створення купюри
num_qubits = 100
quantum_money = WiesnerQuantumMoney(num_qubits)
bases, states = quantum_money.get_qubits()
print("Оригінальні базиси:", bases)
print("Оригінальні стани:", states)

# Банк зберігає інформацію про купюру
bank = Bank(bases, states)

# Власник вимірює купюру
provided_bases, provided_states = owner_measurement(bases, states)
print("Виміряні базиси:", provided_bases)
print("Виміряні стани:", provided_states)

# Перевірка купюри
if bank.verify(provided_bases, provided_states):
    print("Купюра автентична!")
else:
    print("Купюра не автентична!")

Оригінальні базиси: ['x', 'x', '+', 'x', '+', 'x', '+', 'x', 'x', 'x', '+', 'x', '+', '+', 'x', '+', '+', 'x', '+', '+', '+', '+', '+', 'x', '+', '+', 'x', '+', 'x', 'x', 'x', 'x', '+', '+', 'x', 'x', '+', '+', '+', '+', 'x', 'x', 'x', 'x', '+', '+', '+', '+', 'x', 'x', '+', 'x', '+', 'x', 'x', '+', '+', '+', '+', '+', '+', '+', 'x', 'x', '+', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', '+', '+', 'x', 'x', '+', '+', '+', 'x', 'x', '+', 'x', '+', 'x', '+', '+', '+', 'x', 'x', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', '+']
Оригінальні стани: [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]
Виміряні базиси: ['x', 'x', '+', 'x', '+', 'x', '+', '+', 'x', '+', '+', '+', '+', '+', '+', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', '+', '